In [52]:
# import libraries
from pprint import pprint
import importlib

import my_functions as mf

In [54]:
# Get the catalogue from "Our World in Data"
# from owid import catalog
# cat = catalog.find(namespace="covid")

importlib.reload(mf)
mf.disp_catalogue_info(cat)

Row   | Table                          | Dataset                        | Formats                       
------------------------------------------------------------------------------------------
1     | cases_and_deaths_who           | cases_and_deaths_who           | ['feather']                   
2     | cases_deaths                   | cases_deaths                   | ['feather' 'csv']             
3     | chile                          | deaths_vax_status              | ['feather' 'parquet']         
4     | combined                       | combined                       | ['feather' 'parquet']         
5     | compact                        | compact                        | ['feather' 'csv']             
6     | contributions                  | github_stats                   | ['feather' 'parquet']         
7     | country_counts                 | countries_reporting            | ['feather' 'parquet']         
8     | country_counts                 | oxcgrt_policy               

In [55]:
# Load a data set
row_num = 1 # the row from which the data set is defined.
tb = cat.iloc[row_num].load()

# Show results
pprint(list(tb.columns)[0:5]) # first six columns
tb.tail() # show the tail of dataframe

['new_cases', 'total_cases', 'new_deaths', 'total_deaths', 'weekly_cases']


new_cases  total_cases  new_deaths  total_deaths  \
country  date                                                           
Zimbabwe 2025-02-26          0       266410           0          5740   
         2025-02-27          0       266410           0          5740   
         2025-02-28          0       266410           0          5740   
         2025-03-01          0       266410           0          5740   
         2025-03-02          0       266410           0          5740   

                     weekly_cases  weekly_deaths  weekly_pct_growth_cases  \
country  date                                                               
Zimbabwe 2025-02-26             0              0                     <NA>   
         2025-02-27             0              0                     <NA>   
         2025-02-28             0              0                     <NA>   
         2025-03-01             0              0                     <NA>   
         2025-03-02             0              0                     <NA>   

                     weekly_pct_growth_deaths  biweekly_cases  \
country  date                                                   
Zimbabwe 2025-02-26                      <NA>               0   
         2025-02-27                      <NA>               0   
         2025-02-28                      <NA>               0   
         2025-03-01                      <NA>               0   
         2025-03-02                      <NA>               0   

                     biweekly_deaths  ...  cfr_100_cases  cfr_short_term  \
country  date                         ...                                  
Zimbabwe 2025-02-26                0  ...       2.154574            <NA>   
         2025-02-27                0  ...       2.154574            <NA>   
         2025-02-28                0  ...       2.154574            <NA>   
         2025-03-01                0  ...       2.154574            <NA>   
         2025-03-02                0  ...       2.154574            <NA>   

                     days_since_100_total_cases  days_since_5_total_deaths  \
country  date                                                                
Zimbabwe 2025-02-26                        1737                       1819   
         2025-02-27                        1738                       1820   
         2025-02-28                        1739                       1821   
         2025-03-01                        1740                       1822   
         2025-03-02                        1741                       1823   

                     days_since_1_total_cases_per_million  \
country  date                                               
Zimbabwe 2025-02-26                                  1779   
         2025-02-27                                  1780   
         2025-02-28                                  1781   
         2025-03-01                                  1782   
         2025-03-02                                  1783   

                     days_since_0_1_total_deaths_per_million  \
country  date                                                  
Zimbabwe 2025-02-26                                     1819   
         2025-02-27                                     1820   
         2025-02-28                                     1821   
         2025-03-01                                     1822   
         2025-03-02                                     1823   

                     days_since_100_total_cases_and_5m_pop  \
country  date                                                
Zimbabwe 2025-02-26                                   1737   
         2025-02-27                                   1738   
         2025-02-28                                   1739   
         2025-03-01                                   1740   
         2025-03-02                                   1741   

                     total_deaths_last12m  total_deaths_per_100k_last12m  \
country  date                                